### Libraries

In [1]:
import sys
from collections import deque
import heapq
import math


%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
from contextlib import contextmanager
import signal
import time

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

## Classes

### Problem

In [2]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

### Node

In [3]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

### Injection

In [4]:
class injection:
    """Dependency injection of temporary values for global functions/classes/etc.
    E.g., `with injection(DataBase=MockDataBase): ...`"""

    def __init__(self, **kwds):
        self.new = kwds

    def __enter__(self):
        self.old = {v: globals()[v] for v in self.new}
        globals().update(self.new)

    def __exit__(self, type, value, traceback):
        globals().update(self.old)


def memoize(fn, slot=None, maxsize=32):
    """Memoize fn: make it remember the computed value for any argument list.
    If slot is specified, store result in that slot of first argument.
    If slot is false, use lru_cache for caching the values."""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn


def name(obj):
    """Try to find some reasonable name for the object."""
    return (getattr(obj, 'name', 0) or getattr(obj, '__name__', 0) or
            getattr(getattr(obj, '__class__', 0), '__name__', 0) or
            str(obj))


def isnumber(x):
    """Is x a number?"""
    return hasattr(x, '__int__')


def issequence(x):
    """Is x a sequence?"""
    return isinstance(x, collections.abc.Sequence)


def print_table(table, header=None, sep='   ', numfmt='{}'):
    """Print a list of lists as a table, so that columns line up nicely.
    header, if specified, will be printed as the first row.
    numfmt is the format for all numbers; you might want e.g. '{:.2f}'.
    (If you want different formats in different columns,
    don't use print_table.) sep is the separator between columns."""
    justs = ['rjust' if isnumber(x) else 'ljust' for x in table[0]]

    if header:
        table.insert(0, header)

    table = [[numfmt.format(x) if isnumber(x) else x for x in row]
             for row in table]

    sizes = list(map(lambda seq: max(map(len, seq)), list(zip(*[map(str, row) for row in table]))))

    for row in table:
        print(sep.join(getattr(str(x), j)(size) for (j, size, x) in zip(justs, sizes, row)))


def open_data(name, mode='r'):
    aima_root = os.path.dirname(__file__)
    aima_file = os.path.join(aima_root, *['aima-data', name])

    return open(aima_file, mode=mode)


def failure_test(algorithm, tests):
    """Grades the given algorithm based on how many tests it passes.
    Most algorithms have arbitrary output on correct execution, which is difficult
    to check for correctness. On the other hand, a lot of algorithms output something
    particular on fail (for example, False, or None).
    tests is a list with each element in the form: (values, failure_output)."""
    return mean(int(algorithm(x) != y) for x, y in tests)


### Priority Queue

In [5]:
class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)

### EightPuzzle

In [6]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board, where one of the
    squares is a blank. A state is represented as a tuple of length 9, where  element at
    index i represents the tile number  at index i (0 if it's an empty square) """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        super().__init__(initial, goal)

    def find_blank_square(self, state):
        """Return the index of the blank square in a given state"""

        return state.index(0)

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only four possible actions
        in any given state of the environment """

        possible_actions = ['U', 'D', 'L', 'R']
        index_blank_square = self.find_blank_square(state)

        if index_blank_square % 3 == 0:
            possible_actions.remove('L')
        if index_blank_square < 3:
            possible_actions.remove('U')
        if index_blank_square % 3 == 2:
            possible_actions.remove('R')
        if index_blank_square > 5:
            possible_actions.remove('D')

        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """

        # blank is the index of the blank square
        blank = self.find_blank_square(state)
        new_state = list(state)

        delta = {'U': -3, 'D': 3, 'L': -1, 'R': 1}
        neighbor = blank + delta[action]
        new_state[blank], new_state[neighbor] = new_state[neighbor], new_state[blank]

        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """

        return state == self.goal

    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j] != 0:
                    inversion += 1

        return inversion % 2 == 0

    def h(self, node):
        """ Return the heuristic value for a given state. Default heuristic function used is 
        h(n) = number of misplaced tiles """

        return sum(s != g for (s, g) in zip(node.state, self.goal))

### Heuristics

In [7]:
# Heuristics for 8 Puzzle Problem
def misplaced_tile(node):
    state = node.state
    goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)

    misplaced_count = 0

    for i in range(len(state)):
        if state[i] != 0 and state[i] != goal[i]:
            misplaced_count += 1

    return misplaced_count


def manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0
    
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    
    mhd = 0
    
    for i in range(8):
        for j in range(2):
            mhd = abs(index_goal[i][j] - index_state[i][j]) + mhd
    
    return mhd


def linear(node):
    goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])


def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

## Functions

In [8]:
def breadth_first_tree_search(problem):
    """
    [Figure 3.7]
    Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """
    explored = set()
    frontier = deque([Node(problem.initial)])  # FIFO queue

    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        if problem.goal_test(node.state):
            return node, explored
        frontier.extend(node.expand(problem))
    return None


def breadth_first_graph_search(problem):
    """[Figure 3.11]
    Note that this function can be implemented in a
    single line as below:
    return graph_search(problem, FIFOQueue())
    """
    node = Node(problem.initial)
    frontier = deque([node])
    explored = set()
    if problem.goal_test(node.state):
        return node, explored
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child, explored
                frontier.append(child)
    return None, explored


def depth_limited_search(problem, limit=50):
    """[Figure 3.17]"""
    explored = set()
    def recursive_dls(node, problem, limit):
        explored.add(node.state)
        if problem.goal_test(node.state):
            return node, explored
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit)


def iterative_deepening_search(problem):
    """[Figure 3.18]"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result
        

def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node, explored, frontier
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None, explored, frontier

        
def astar_search_1(problem, h=misplaced_tile, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    solution, explored, frontier = best_first_graph_search(problem, lambda n: n.path_cost + h(n), display) 
    return solution, explored, frontier

def astar_search_2(problem, h=manhattan, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    solution, explored, frontier = best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)
    return solution, explored, frontier

def astar_search_3(problem, h=max_heuristic, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    solution, explored, frontier = best_first_graph_search(problem, lambda n: n.path_cost + h(n), display) 
    return solution, explored, frontier

## Part 1: (40 pts)

In [12]:
class TimeoutException(Exception):
    pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

def time_config(total_seconds):
    # Calculate minutes, seconds, and microseconds
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    microseconds = int((total_seconds - int(total_seconds)) * 1_000_000)
    
    if (minutes == 0) and (seconds == 0):
        time_taken = f"{microseconds} microSec."
    
    elif minutes == 0:
        time_taken = f"{seconds} sec {microseconds} microSec."
    
    else:
        time_taken = f"{minutes} min {seconds} sec {microseconds} microSec."
        
    return time_taken

# define function to get the required output
def func_output(algo, algorithm_name, problem, heuristic=None, display=True):

    # start timing
    start_time = time.perf_counter()
    

    if algorithm_name == "BFGS" or algorithm_name == "BFTS" or algorithm_name == "IDS":
#         print(algorithm_name)
        solution, explored = algo(problem)
        frontier = []
    else:
        solution, explored, frontier = algo(problem)


    # required output
    seq_actions = solution.solution()
    path = solution.path()
    path_lenght = len(path)
    tot_nodes_generated = len(explored) + len(frontier)

    # edn timing
    end_time = time.perf_counter()

    ## total time taken
    total_seconds = end_time - start_time

    # Calculate minutes, seconds, and microseconds
    time_taken = time_config(total_seconds)

    return print(f"Total nodes generated: {tot_nodes_generated}\n"
      f"Total Time Taken: {time_taken}\n"
      f"Path length: {path_lenght}\n"
      f"Path: {''.join(seq_actions)}")
    
def puzzle_8_solver(file_path, algorithm):
    try:
        with time_limit(900):  # 900 seconds = 15 minutes    
            
            # read files in
            with open(file_path, 'r') as file:
                puzzle_raw = file.read().split()
            puzzle_int = tuple(int(x if x != '_' else '0') for x in puzzle_raw)

            # fit puzzle in
            puzzle = EightPuzzle(puzzle_int)
#             puzzle = EightPuzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

            # check for solvability
            is_solvable = puzzle.check_solvability(puzzle_int)

            if is_solvable == False:
                print("Problem is not solvable.")
                return None 

            # dictionary to map algorithm names to their corresponding functions
            algo_dict = {
                'BFGS': breadth_first_graph_search,
                'BFTS': breadth_first_tree_search,
                'IDS': iterative_deepening_search,
                'h1': astar_search_1,
                'h2': astar_search_2,
                'h3': astar_search_3
            }


            if algorithm in algo_dict:
                return func_output(algo_dict[algorithm], algorithm, puzzle)
            else:
                print(f"Algorithm {algorithm} is not recognized. The available algorithms are: BFGS, BFTS, IDS, h1, h2, h3")

    except TimeoutException as e:
        print("Total nodes generated: Timed out")
        print("Total Time Taken: >15 min")
        print("Path length: Timed out")
        print("Path: Timed out")

#### breadth_first_graph_search

In [13]:
puzzle_8_solver("../../Part2/S5.txt", "BFGS")

Total nodes generated: 54
Total Time Taken: 766 microSec.
Path length: 7
Path: LURDDR


#### breadth_first_tree_search

In [15]:
puzzle_8_solver("../../Part2/S5.txt", "BFTS")

Total nodes generated: 103
Total Time Taken: 5946 microSec.
Path length: 7
Path: LURDDR


#### iterative_deepening_search

In [16]:
puzzle_8_solver("../../Part2/S5.txt", "IDS")

Total nodes generated: 88
Total Time Taken: 2676 microSec.
Path length: 7
Path: LURDDR


#### astar_search_1

In [17]:
puzzle_8_solver("../../Part2/S5.txt", "h1")

Total nodes generated: 19
Total Time Taken: 204 microSec.
Path length: 7
Path: LURDDR


#### astar_search_2

In [18]:
puzzle_8_solver("../../Part2/S5.txt", "h2")

Total nodes generated: 20
Total Time Taken: 376 microSec.
Path length: 7
Path: LURDDR


#### astar_search_3

In [19]:
puzzle_8_solver("../../Part2/S5.txt", "h3")

Total nodes generated: 20
Total Time Taken: 380 microSec.
Path length: 7
Path: LURDDR


## Part 2: (20 pts)

In [20]:
part_2_files = ["../../Part2/S1.txt", "../../Part2/S2.txt", "../../Part2/S3.txt", "../../Part2/S4.txt", "../../Part2/S5.txt"]

for file in part_2_files:
    print(file)

../../Part2/S1.txt
../../Part2/S2.txt
../../Part2/S3.txt
../../Part2/S4.txt
../../Part2/S5.txt


### A* using Manhattam Distance heuristic

In [27]:
print("A* using Manhattam Distance Heuristic\n")
algorithm = "h2"
for file in part_2_files:
    print("Solving problem for file: ", file)
    puzzle_8_solver(file, algorithm)
    print(" ")

A* using Manhattam Distance as Heuristic

Solving problem for file : ../Part2/S1.txt
Total nodes generated: 3188
Total Time Taken: 851894 microSec.
Path length: 25
Path: UURDDRULLDRRULLURRDLLDRR
 
Solving problem for file : ../Part2/S2.txt
Total nodes generated: 320
Total Time Taken: 12215 microSec.
Path length: 21
Path: UURRDLDRULLURRDLLDRR
 
Solving problem for file : ../Part2/S3.txt
Problem is not solvable.
 
Solving problem for file : ../Part2/S4.txt
Total nodes generated: 25947
Total Time Taken: 1 min 21 sec 516981 microSec.
Path length: 32
Path: RUULLDDRRULLDRRUULDRULLDDRRULDR
 
Solving problem for file : ../Part2/S5.txt
Total nodes generated: 20
Total Time Taken: 426 microSec.
Path length: 7
Path: LURDDR
 


### A* using Max heuristic

In [28]:
print("A* using Max Heuristic \n")
algorithm = "h3"
for file in part_2_files:
    print("Solving problem for file: ", file)
    puzzle_8_solver(file, algorithm)
    print(" ")

A* using Max Heuristic as Heuristic

Solving problem for file : ../Part2/S1.txt
Total nodes generated: 3188
Total Time Taken: 894952 microSec.
Path length: 25
Path: UURDDRULLDRRULLURRDLLDRR
 
Solving problem for file : ../Part2/S2.txt
Total nodes generated: 320
Total Time Taken: 12725 microSec.
Path length: 21
Path: UURRDLDRULLURRDLLDRR
 
Solving problem for file : ../Part2/S3.txt
Problem is not solvable.
 
Solving problem for file : ../Part2/S4.txt
Total nodes generated: 25947
Total Time Taken: 1 min 11 sec 309168 microSec.
Path length: 32
Path: RUULLDDRRULLDRRUULDRULLDDRRULDR
 
Solving problem for file : ../Part2/S5.txt
Total nodes generated: 20
Total Time Taken: 438 microSec.
Path length: 7
Path: LURDDR
 


### A* using Misplaced Tile heuristic

In [21]:
print("A* using Misplaced Tile heuristic\n")
algorithm = "h1"
for file in part_2_files:
    print("Solving problem for file: ", file)
    puzzle_8_solver(file, algorithm)
    print(" ")

A* using default Heuristic

Solving problem for file:  ../../Part2/S1.txt
Total nodes generated: 18818
Total Time Taken: 31 sec 681916 microSec.
Path length: 25
Path: UURDDRULLDRRULLURRDLLDRR
 
Solving problem for file:  ../../Part2/S2.txt
Total nodes generated: 2918
Total Time Taken: 734186 microSec.
Path length: 21
Path: UURRDLDRULLURRDLLDRR
 
Solving problem for file:  ../../Part2/S3.txt
Problem is not solvable.
 
Solving problem for file:  ../../Part2/S4.txt
Total nodes generated: Timed out
Total Time Taken: >15 min
Path length: Timed out
Path: Timed out
 
Solving problem for file:  ../../Part2/S5.txt
Total nodes generated: 19
Total Time Taken: 344 microSec.
Path length: 7
Path: LURDDR
 


### Breadth First Graph Search

In [31]:
print("Breadth First Graph Search\n")
algorithm = "BFGS"
for file in part_2_files:
    print("Solving problem for file: ", file)
    puzzle_8_solver(file, algorithm)
    print(" ")

Breadth First Graph Search

Solving problem for file:  ../Part2/S1.txt
Total nodes generated: 97527
Total Time Taken: 6 min 43 sec 57791 microSec.
Path length: 25
Path: UURDDRULLDRRULLURRDLLDRR
 
Solving problem for file:  ../Part2/S2.txt
Total nodes generated: 29053
Total Time Taken: 53 sec 187489 microSec.
Path length: 21
Path: UURRDLDRULLURRDLLDRR
 
Solving problem for file:  ../Part2/S3.txt
Problem is not solvable.
 
Solving problem for file:  ../Part2/S4.txt
Total nodes generated: 181347
Total Time Taken: 11 min 54 sec 503927 microSec.
Path length: 32
Path: UULDDRRUULDLDRRUULDLDRRUULLDDRR
 
Solving problem for file:  ../Part2/S5.txt
Total nodes generated: 54
Total Time Taken: 1061 microSec.
Path length: 7
Path: LURDDR
 


### Iterative Deepening Search

In [32]:
print("Iterative Deepening Search\n")
algorithm = "IDS"
for file in part_2_files:
    print("Solving problem for file: ", file)
    puzzle_8_solver(file, algorithm)
    print(" ")

Iterative Deepening Search

Solving problem for file:  ../Part2/S1.txt
Total nodes generated: Timed out
Total Time Taken: >15 min
Path length: Timed out
Path: Timed out
 
Solving problem for file:  ../Part2/S2.txt
Total nodes generated: Timed out
Total Time Taken: >15 min
Path length: Timed out
Path: Timed out
 
Solving problem for file:  ../Part2/S3.txt
Problem is not solvable.
 
Solving problem for file:  ../Part2/S4.txt
Total nodes generated: Timed out
Total Time Taken: >15 min
Path length: Timed out
Path: Timed out
 
Solving problem for file:  ../Part2/S5.txt
Total nodes generated: 88
Total Time Taken: 7004 microSec.
Path length: 7
Path: LURDDR
 


In [ ]:
# read files in
with open("../Part2/S2.txt", 'r') as file:
    puzzle_s1_raw = file.read().split()
    puzzle_s1 = tuple(int(x if x != '_' else '0') for x in puzzle_s1_raw)

puzzle = EightPuzzle(puzzle_s1)
puzzle.initial

In [ ]:
def depth_limited_search(problem, limit=50):
    explored = set()
    def recursive_dls(node, problem, limit, explored):
        # Before lose, make sure state not before recast
        if problem.goal_test(node.state):
            return node, explored
        elif limit == 0:
            return 'cutoff', None
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                if child.state not in explored:
                    explored.add(child.state)
                    result, _ = recursive_dls(child, problem, limit - 1, explored)
                    if result == 'cutoff':
                        cutoff_occurred = True
                    elif result is not None:
                        return result, explored
            return ('cutoff' if cutoff_occurred else None), explored



    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit, explored)

def iterative_deepening_search(problem):
    for depth in range(sys.maxsize):
        # Problem context pre and position command
        result, explored = depth_limited_search(problem, depth)
        # Extract depth_limited_search callback, loop, and properly handle churn or `cutoff` values
        if result != 'cutoff':  # Re-template correct finishing mode callback or review if fault in search hand pattern
            return result, explored


In [ ]:
def breadth_first_graph_search(problem):
    node = Node(problem.initial)
    frontier = deque([node])
    explored = set()
    if problem.goal_test(node.state):
        return node, explored
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child, explored
                frontier.append(child)
    return None, explored

In [ ]:
def depth_limited_search(problem, limit=50):
    """[Figure 3.17]"""
    explored = set()
    def recursive_dls(node, problem, limit):
        explored.add(node.state)
        if problem.goal_test(node.state):
            return node, explored
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit)


def iterative_deepening_search(problem):
    """[Figure 3.18]"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result

In [ ]:
def depth_limited_search(problem, limit=50, explored=None):
    if explored is None:
        explored = set()
    
    def recursive_dls(node, problem, limit, depth=0):
        explored.add(node.state)
        if problem.goal_test(node.state):
            return node, len(explored)  # Additionally, we can show the bookkeeping of the local copy
        elif limit == 0:
            return 'cutoff', None
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                if child.state not in explored:  # More traditional halting, with apt dynamics
                    result, _ = recursive_dls(child, problem, limit - 1, depth + 1)
                    if result == 'cutoff':
                        cutoff_occurred = True
                    elif result is not None:
                        return result, len(explored)
            return ('cutoff', None) if cutoff_occurred else (None, None)

    result, final_length = recursive_dls(Node(problem.initial), problem, limit)
    return result, final_length, explored  # Freely, the order could reflect the ending or restart

def iterative_deepening_search(problem):
    total_nodes_visited = 0
    explored_across_depths = set()  # A way to maintain state across the graph
    for depth in range(sys.maxsize):
        result, final_length, explored = depth_limited_search(problem, depth, explored_across_depths)
        total_nodes_visited += final_length if final_length is not None else 0
        if result != 'cutoff':
            return result, final_length  # Inclusive of the very opulent set

# Mindful, Node and the actual criteria of actionability are still receptive to the directorship of the story in such complexity.


In [ ]:
def depth_limited_search(problem, limit=50):
    explored = set()
    node_counter = {'count': 0}  

    def recursive_dls(node, problem, limit, depth=0):
        node_counter['count'] += 1  
        if node.state not in explored or depth < explored[node.state]:
            explored[node.state] = depth
            if problem.goal_test(node.state):
                return node, node_counter['count']  
            elif limit == 0:
                return 'cutoff', None
            else:
                cutoff_occurred = False
                for child in node.expand(problem):
                    result, _ = recursive_dls(child, problem, limit - 1, depth + 1)
                    if result == 'cutoff':
                        cutoff_occurred = True
                    elif result is not None:
                        return result, node_counter['count']
                return ('cutoff', None) if cutoff_occurred else (None, None)
        return None, None

    
    explored = {}
    
    result, count = recursive_dls(Node(problem.initial), problem, limit)
    return result, count

def iterative_deepening_search(problem):
    total_nodes_visited = 0
    for depth in range(sys.maxsize):
        result, count = depth_limited_search(problem, depth)
        if count:
            total_nodes_visited += count
        if result != 'cutoff':
            return result, total_nodes_visited


In [ ]:
solution, explored = iterative_deepening_search(puzzle)

In [ ]:
len(solution.path())

In [ ]:
path = solution.path()
path_lenght = len(path)
path_lenght

In [ ]:
solution, explored, frontier = astar_search_2(puzzle, manhattan, display=True)
path = solution.path()
path_lenght = len(path)
path_lenght

In [ ]:
len(frontier)